# 머신러닝 with Python 
## 09 앙상블 학습 배깅(Bootstrap aggregating) 실습- wine data set

##### 배깅 알고리즘을 이용하여 나이브 베이즈 모델을 활용, 와인 데이터를 분류하는 모형을 만들어본다.

책 p.267~ 

---

하단부에서는 분류기의 개수에 따른 성능의 변화를 실험해 보았다.

> 배운점1: 분류기 개수가 성능에 영향을 미친다. 분류기 개수가 늘어날수록 일정 수준까지 성능이 떨어지고 멈췄다.

> 배운점2: 개수가 적고 성능이 좋다고 하더라도 해당 모델이 실제로 unseen 데이터 대응이 좋은지는 미지수이다.

### 데이터 불러오기

In [1]:
from sklearn import datasets

In [2]:
raw_wine = datasets.load_wine() # 와인 데이터 가져오기

In [3]:
# 데이터 살펴보기
raw_wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [4]:
raw_wine.feature_names      # 데이터 셋 내 피처 이름들

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

In [5]:
raw_wine.target_names

array(['class_0', 'class_1', 'class_2'], dtype='<U7')

### 피처, 타깃 데이터 지정

In [6]:
X = raw_wine.data
y = raw_wine.target

### 트레이닝/테스트 데이터 분할

In [7]:
from sklearn.model_selection import train_test_split                 # 분할을 위해 필요한 함수
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0)    # 분리. randomstate는 고정

### 데이터 표준화

In [8]:
from sklearn.preprocessing import StandardScaler         # 데이터 표준화를 위한 함수
std_scale = StandardScaler()                             # 표준화 스케일러 지정
std_scale.fit(X_tn)                                      # 트레이닝 피처를 기준으로 표준화를 적합 시도

X_tn_std = std_scale.transform(X_tn) 
X_te_std = std_scale.transform(X_te)                     # 트레인, 테스트 데이터 각각 적합시킨 표준화에 맞게 변형

### 데이터 학습

In [10]:
from sklearn.naive_bayes import GaussianNB               # 가우시안 나이브 베이즈 개별 분류기
from sklearn.ensemble import BaggingClassifier           # 배깅 앙상블을 사용하여 분류

# 배깅을 활용한 분류기 생성
clf_bagging_gnb = BaggingClassifier(base_estimator = GaussianNB(),  # base_estimator : 개별 학습기 입력
                                    n_estimators = 10,              # n_estimator : 개별 학습기의 개수
                                    random_state = 0)               # 고정

clf_bagging_gnb.fit(X_tn_std, y_tn)                                 # 적합시키기

BaggingClassifier(base_estimator=GaussianNB(), random_state=0)

### 데이터 예측

In [11]:
pred_bagging_gnb = clf_bagging_gnb.predict(X_te_std)                # std된 테스트 피처 데이터를 넣고 실행하여 결과 확인 
print(pred_bagging_gnb)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [12]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_bagging_gnb)                   # 실제값과 예측값을 넣음
print(accuracy)

0.9555555555555556


### confusion matrix 확인

In [13]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_bagging_gnb)
print(conf_matrix)

[[16  0  0]
 [ 1 19  1]
 [ 0  0  8]]


### 분류 리포트 확인

In [14]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_bagging_gnb)
print(class_report)

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.90      0.95        21
           2       0.89      1.00      0.94         8

    accuracy                           0.96        45
   macro avg       0.94      0.97      0.95        45
weighted avg       0.96      0.96      0.96        45



### 결과

적당한 정확도군요. 그럼 여기서 의문이 드는 것이 분류기의 개수와 분류기의 종류인데 종류는 노트북을 따로 만들기로 하고 분류기의 개수를 변경시켜봅시다.

# 실험: 분류기의 개수와 성능

## 실험1: 분류기가 10개에서 5개로 반토막이 났을 때 

### 데이터 학습

In [15]:
from sklearn.naive_bayes import GaussianNB               # 가우시안 나이브 베이즈 개별 분류기
from sklearn.ensemble import BaggingClassifier           # 배깅 앙상블을 사용하여 분류

# 배깅을 활용한 분류기 생성
clf_bagging_gnb = BaggingClassifier(base_estimator = GaussianNB(),  # base_estimator : 개별 학습기 입력
                                    n_estimators = 5,               # n_estimator : 개별 학습기의 개수
                                    random_state = 0)               # 고정

clf_bagging_gnb.fit(X_tn_std, y_tn)                                 # 적합시키기

BaggingClassifier(base_estimator=GaussianNB(), n_estimators=5, random_state=0)

### 데이터 예측

In [16]:
pred_bagging_gnb = clf_bagging_gnb.predict(X_te_std)                # std된 테스트 피처 데이터를 넣고 실행하여 결과 확인 
print(pred_bagging_gnb)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [17]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_bagging_gnb)                   # 실제값과 예측값을 넣음
print(accuracy)

0.9777777777777777


### confusion matrix 확인

In [18]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_bagging_gnb)
print(conf_matrix)

[[16  0  0]
 [ 1 20  0]
 [ 0  0  8]]


### 분류 리포트 확인

In [19]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_bagging_gnb)
print(class_report)

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.95      0.98        21
           2       1.00      1.00      1.00         8

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



### 결과

분류기의 개수가 반으로 줄어들었는데 성능이 오히려 올라갔다. 클래스 1의 분류가 더 정확해졌다.

분류기의 개수가 많을 수록 정확도가 떨어질 수 있다.

여기서 생각해 볼 수 있는 가설은

1. 분류기의 개수에 따라 과적합의 문제가 일어날 수 있다. 오버피팅이 된건지 아닌지는 알 수 없다.

2. 분류기의 개수가 많을 수록 꼭 수치적인 성능이 좋아지는 것은 아니다.

---

이어서 실험을 분류기의 개수를 더 줄여본다.

## 실험2: 분류기가 5개에서 2개로 극단적으로 줄었을 때

### 데이터 학습

In [20]:
from sklearn.naive_bayes import GaussianNB               # 가우시안 나이브 베이즈 개별 분류기
from sklearn.ensemble import BaggingClassifier           # 배깅 앙상블을 사용하여 분류

# 배깅을 활용한 분류기 생성
clf_bagging_gnb = BaggingClassifier(base_estimator = GaussianNB(),  # base_estimator : 개별 학습기 입력
                                    n_estimators = 2,               # n_estimator : 개별 학습기의 개수
                                    random_state = 0)               # 고정

clf_bagging_gnb.fit(X_tn_std, y_tn)                                 # 적합시키기

BaggingClassifier(base_estimator=GaussianNB(), n_estimators=2, random_state=0)

### 데이터 예측

In [21]:
pred_bagging_gnb = clf_bagging_gnb.predict(X_te_std)                # std된 테스트 피처 데이터를 넣고 실행하여 결과 확인 
print(pred_bagging_gnb)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [22]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_bagging_gnb)                   # 실제값과 예측값을 넣음
print(accuracy)

1.0


### confusion matrix 확인

In [23]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_bagging_gnb)
print(conf_matrix)

[[16  0  0]
 [ 0 21  0]
 [ 0  0  8]]


### 분류 리포트 확인

In [24]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_bagging_gnb)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



### 결과

분류기의 개수가 더 줄었더니 정확도가 1.0이 되어버렸다.

실제로 개수가 줄어듦에 따라 성능이 높아진건 좋지만, 성능이 높다는 것은 그만큼 unseen 데이터에 대한 정확도가 낮을 가능성이 높다는 생각이 든다.

---

실험 분류기의 개수를 2~10개로 돌려본다.

## 실험3: 분류기 개수 2~10개 각각 성능 체크해보기

In [26]:
from sklearn.naive_bayes import GaussianNB               # 가우시안 나이브 베이즈 개별 분류기
from sklearn.ensemble import BaggingClassifier           # 배깅 앙상블을 사용하여 분류
from sklearn.metrics import accuracy_score               # 정확도 테스트 함수
from sklearn.metrics import classification_report        # 분류 리포트 확인을 위한 함수


for i in range(2, 11):
    # 배깅을 활용한 분류기 생성
    clf_bagging_gnb = BaggingClassifier(base_estimator = GaussianNB(), 
                                        n_estimators = i,               # n_estimator : 개별 학습기의 개수
                                        random_state = 0)               # 고정
    
    # 데이터 학습
    clf_bagging_gnb.fit(X_tn_std, y_tn)

    # 데이터 예측
    pred_bagging_gnb = clf_bagging_gnb.predict(X_te_std) 
    
    # 데이터 정확도 평가
    accuracy = accuracy_score(y_te, pred_bagging_gnb)
    
    # 분류 리포트 확인
    class_report = classification_report(y_te, pred_bagging_gnb)
    
    # 모델 상황 확인
    
    print("모델 분류기 개수: ", i )
    print("모델 정확도: ", accuracy)
    print("모델 분류 리포트")
    print(class_report)
    print("========================================================")

모델 분류기 개수:  2
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

모델 분류기 개수:  3
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

모델 분류기 개수:  4
모델 정확도:  0.9777777777777777
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
   

 ### 결과
 
 실험 결과를 보면 
 
 모델 분류기 개수가 [2,3], [4,5,6,7],[8,9,10] 이렇게 정확도가 같게 나왔다. 실제로 클래스별 f1-score도 같게 나왔다.
 
 분류기의 개수가 늘어날수록 정확도가 떨어지는 이유에 대해서 제대로 할 수는 없을까?
 
 그럼 10보다 클 때는 어떻게 될까?

## 실험3: 분류기가 11개 이상일 떄

In [27]:
from sklearn.naive_bayes import GaussianNB               # 가우시안 나이브 베이즈 개별 분류기
from sklearn.ensemble import BaggingClassifier           # 배깅 앙상블을 사용하여 분류
from sklearn.metrics import accuracy_score               # 정확도 테스트 함수
from sklearn.metrics import classification_report        # 분류 리포트 확인을 위한 함수


for i in range(11, 21):
    # 배깅을 활용한 분류기 생성
    clf_bagging_gnb = BaggingClassifier(base_estimator = GaussianNB(), 
                                        n_estimators = i,               # n_estimator : 개별 학습기의 개수
                                        random_state = 0)               # 고정
    
    # 데이터 학습
    clf_bagging_gnb.fit(X_tn_std, y_tn)

    # 데이터 예측
    pred_bagging_gnb = clf_bagging_gnb.predict(X_te_std) 
    
    # 데이터 정확도 평가
    accuracy = accuracy_score(y_te, pred_bagging_gnb)
    
    # 분류 리포트 확인
    class_report = classification_report(y_te, pred_bagging_gnb)
    
    # 모델 상황 확인
    
    print("모델 분류기 개수: ", i )
    print("모델 정확도: ", accuracy)
    print("모델 분류 리포트")
    print(class_report)
    print("========================================================")

모델 분류기 개수:  11
모델 정확도:  0.9555555555555556
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.90      0.95        21
           2       0.89      1.00      0.94         8

    accuracy                           0.96        45
   macro avg       0.94      0.97      0.95        45
weighted avg       0.96      0.96      0.96        45

모델 분류기 개수:  12
모델 정확도:  0.9555555555555556
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.90      0.95        21
           2       0.89      1.00      0.94         8

    accuracy                           0.96        45
   macro avg       0.94      0.97      0.95        45
weighted avg       0.96      0.96      0.96        45

모델 분류기 개수:  13
모델 정확도:  0.9555555555555556
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.94 

### 결과

아주 재밌는 결과가 나왔다. 11이상이 되어도 더 이상 정확도가 떨어지지 않았다.

그럼 이 앙상블 모델에서는 가장 최적의 결과를 무엇으로 생각할까?

정확도가 가장 높으면서, 분류기의 개수가 적으면 컴퓨팅 파워를 적게 소모할 것이니 가장 적은 분류기의 개수여야 할 것이다.

그렇다면 분류기의 개수가 2개인게 좋은 것일까? 

다른 분류기 종류에서도 이런 결과가 나올까?

--- 

이어서 다른 분류기로 wine데이터를 분류해보아야겠다.